In [1]:
import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')

In [3]:
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))

In [4]:
bc = nx.betweenness_centrality(g)
# cc = nx.closeness_centrality(g)
# ec = nx.eigenvector_centrality(g, max_iter=1000)

In [5]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'ConnAmnt': [g.nodes[i]['amount'] for i in g.nodes],
    'Intermediation (r)': [i for i in bc.values()],
#     'closeness': [i for i in cc.values()],
#     'eigenvector': [i for i in ec.values()]
})

In [6]:
char_infos = pd.read_csv('dataset/charcters_stats.csv', index_col=0)

In [7]:
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [8]:
import numpy as np
semi_df = semi_df.replace('nan', np.nan)
semi_df = semi_df[semi_df['ConnAmnt'].notna()]
semi_df['ConnAmnt'] = pd.to_numeric(semi_df['ConnAmnt'])

In [9]:
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')

In [10]:
# import seaborn as sns
# sns.histplot(semi_df, x='Strength', common_norm = True, stat='probability', cumulative=True,bins=[i for i in range(100)])

In [11]:
semi_df['Alignment'].value_counts()
semi_df = semi_df[semi_df['Alignment'] != 'neutral']

In [12]:
semi_df = semi_df.replace(['good', 'bad'], [1, 0])

In [13]:
import statsmodels.api as sm

model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Power', 'Alignment']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Intermediation (r)   R-squared (uncentered):                   0.326
Model:                            OLS   Adj. R-squared (uncentered):              0.310
Method:                 Least Squares   F-statistic:                              19.62
Date:                Fri, 13 Nov 2020   Prob (F-statistic):                    3.48e-13
Time:                        17:48:52   Log-Likelihood:                          621.31
No. Observations:                 166   AIC:                                     -1235.
Df Residuals:                     162   BIC:                                     -1222.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence  6.096e-05   2.02e-05      3.013      0.003     2.1e-05       0.000
Strength      1.416e-06    1.6e-05      0.088      0.930   -3.02e-05    3.31e-05
Power        -2.282e-05   2.03e-05     -1.124      0.263   -6.29e-05    1.73e-05
Alignment        0.0018      0.001      2.281      0.024       0.000       0.003
==============================================================================
Omnibus:                      141.251   Durbin-Watson:                   1.111
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1475.718
Skew:                           3.240   Prob(JB):                         0.00
Kurtosis:                      16.090   Cond. No.                         168.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""